In [1]:
Email = "meharaliaftab20@gmail.com"
passwd = "BITBASH20"

In [2]:
import csv
import time
from random import uniform
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException,WebDriverException,NoSuchElementException
import os

In [5]:
def type_like_human(element, text):
    for char in text:
        element.send_keys(char)

def clear_input_like_human(element):
    input_text = element.get_attribute('value')
    for _ in range(len(input_text)):
        # time.sleep(random.uniform(0.1, 0.2))
        element.send_keys(Keys.BACKSPACE)

In [19]:
def openprofile(profileid):
    incogniton_profile_id = profileid

    incogniton_url = 'http://127.0.0.1:25000/automation/launch/python/'
    data = {
        "profileID": incogniton_profile_id,
        "customArgs": "--disable-notifications"
    }
    try:
        resp = requests.post(incogniton_url, json=data)
        resp.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        print(f"Browser opened successfully with profile ID: {incogniton_profile_id}")
        incoming_json = resp.json()
        incoming_url = incoming_json.get('url')

        # Creating a remote WebDriver instance
        options = webdriver.ChromeOptions()
        # Add any additional options you may need
        driver = webdriver.Remote(command_executor=incoming_url, options=options)
        driver.maximize_window()
        time.sleep(5)
        
        # Perform actions with the WebDriver
        driver.get('https://pk.linkedin.com/')
        time.sleep(3)
        print(f"{driver.title} is open !!!!")
        return driver
    except requests.exceptions.RequestException as e:
        print(f"Failed to open the browser: {e}")

In [183]:
profileid='bbd273f6-08db-4cf3-95c9-88cfa4ee9060'
message = "Hey! This is test message."
max_attempts=3
for attempt in range(max_attempts):
    try:
        driver = openprofile(profileid)
        if driver:
            break
    except Exception as e:
        print(f"Attempt {attempt + 1} failed: {str(e)}")

Browser opened successfully with profile ID: bbd273f6-08db-4cf3-95c9-88cfa4ee9060
Feed | LinkedIn is open !!!!


In [96]:
emailfieldxpath = '(//*[@class="text-color-text font-sans text-md outline-0 bg-color-transparent grow"])[1]'
passwordxpath = '(//*[@class="text-color-text font-sans text-md outline-0 bg-color-transparent grow"])[2]'
submitxpath = "//button[@type='submit']"

searchbtnxpath = '//button[@class="search-global-typeahead__collapsed-search-button"]'
searchboxxpath = '//input[@class="search-global-typeahead__input"]'

peoplebtnxpath = "(//*[text()='People'])[1]"

container_xpath = '//*[@class="search-results-container"]'
row_xpath = ".//li[contains(@class, 'reusable-search__result-container')]"
username_xpath = "//a[contains(@class, 'app-aware-link')]//span[@dir='ltr']//span[@aria-hidden='true']"
designation_xpath = '//*[@class="entity-result__primary-subtitle t-14 t-black t-normal"]'
location_xpath = '//*[@class="entity-result__secondary-subtitle t-14 t-normal"]'
current_xpath = "//p[contains(@class, 'entity-result__summary') and contains(@class, 'entity-result__summary--2-lines')]"

In [21]:
def autologin(driver , Email , passwd):
    emailfield = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH , emailfieldxpath)))
    if emailfield.get_attribute("value"):
        clear_input_like_human(emailfield)  
        type_like_human(emailfield, Email)
    else:
        emailfield.click()
        type_like_human(emailfield, Email)
    
    time.sleep(uniform(1.5 , 2.5))

    passwordfield = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH , passwordxpath)))
    if passwordfield.get_attribute("value"):
        clear_input_like_human(passwordfield)  
        type_like_human(passwordfield, passwd)
    else:
        emailfield.click()
        type_like_human(passwordfield, passwd)
    
    time.sleep(uniform(1.5 , 2.5))   
    
    action = ActionChains(driver)
    action.send_keys(Keys.ENTER).perform()

In [144]:
def autosearch(driver , name):
    searchbtn =WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="global-nav-search"]')))
    searchbtn.click()
    time.sleep(uniform(1.5,2.5))
    searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,searchboxxpath)))
    if searchbox.get_attribute("value"):
        clear_input_like_human(searchbox)  
        type_like_human(searchbox, name)
    else:
        type_like_human(searchbox, name)
    
    time.sleep(2)
    searchbox.send_keys(Keys.ENTER)

In [147]:
name=["CEO"]
autosearch(driver , name)

In [23]:
def findpeople(driver):
    peoplebtn =WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, peoplebtnxpath)))
    peoplebtn.click()

In [24]:
def save_info(name , data_list):
    with open(f'userfiles/{name}.txt', mode='a', newline="" , encoding="utf-8") as f:
        for data in data_list:
            f.write('username=' + data['username'] + ',\n')
            f.write('designations=' + data['designation'] + ',\n')
            f.write('location=' + data['location'] + ',\n')
            f.write('current=' + data['current'] + ',\n\n')

    print("Data has been saved to txt files.")

In [25]:
def send_message(driver , message):
    try:
        action = ActionChains(driver)
        msgbtn = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH , "(//button[contains(., 'Message')])[2]")))
        action.move_to_element(msgbtn)
        time.sleep(1)
        msgbtn.click()
        print("Message Btn Clicked")
        time.sleep(5)
        msgarea = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH , "//div[contains(@class, 'msg-form__msg-content-container') and contains(@class, 'msg-form__message-texteditor')]//div[contains(@class, 'msg-form__msg-content-container--scrollable')]//div[contains(@class, 'msg-form__contenteditable')]")))
        if msgarea:
            action.move_to_element(msgarea)
            time.sleep(1)
            msgarea.click()
            print("Message Area Clicked")

            print("Type Message...")
            type_like_human(msgarea , message)
            sendbtn = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH ,"//button[text()='Send']")))
            action.move_to_element(sendbtn)
            time.sleep(3)
            # sendbtn.click()

            closebtn = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH ,"//button[@class='msg-overlay-bubble-header__control artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view']")))
            action.move_to_element(closebtn)
            time.sleep(1)
            closebtn.click()
            time.sleep(2)

            discardbtn = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH ,"//button[normalize-space(.)='Discard']")))
            action.move_to_element(discardbtn)
            time.sleep(1)
            discardbtn.click()
            time.sleep(2)
    except:
        print("Error Not found elements!!!")
        closebtn = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH ,"//button[@class='msg-overlay-bubble-header__control artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view']")))
        action.move_to_element(closebtn)
        time.sleep(1)
        closebtn.click()
        time.sleep(2)

In [26]:
def find_next(driver):
    action = ActionChains(driver)
    nextbtn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Next')]")))
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});",nextbtn)
    action.move_to_element(nextbtn)
    time.sleep(1)
    nextbtn.click()

In [134]:
def save_user_in_csv(file_path, username):
    with open(file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow([username])
        print(f"User {username} save in Csv...")

In [133]:
save_user_in_csv("./UsersData/username.csv", "farhad")

In [155]:
def username_exists(username, alreadysendmsg_filepath):
    # Check if the username exists in alreadysendmsg.csv
    with open(alreadysendmsg_filepath, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if username == row:
                return True
    return False

In [156]:
def main2(driver, message, usernamefilepath):
    usernames = []
    with open(usernamefilepath, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            usernames.append(row)
    
    for username in usernames:
        if username_exists(username, "./Already/alreadysendmsg.csv"):
            print(f"Username '{username}' already exists in alreadysendmsg.csv. Skipping...")
            continue
        else:
            print(f"Sending message to '{username}'...")
            time.sleep(2)
            autosearch(driver , username)
            break
    return usernames

In [157]:
usernamefilepath = "./UsersData/username.csv"
main2(driver, message, usernamefilepath)

Username '['Ahmad Chaudhary']' already exists in alreadysendmsg.csv. Skipping...
Sending message to '['Sandeep Soni']'...


[['Ahmad Chaudhary'],
 ['Sandeep Soni'],
 ['Bill Fisher'],
 ['Umer Farooq'],
 ['Ritesh Satia'],
 ['Mauro Gomide'],
 [],
 [],
 [],
 [],
 []]

In [108]:
def scrapdata(driver):
    datalist = []
    count = 2
    container = driver.find_element(By.XPATH, "//*[@class='search-results-container']")
    time.sleep(1)
    rows = container.find_elements(By.XPATH, ".//li[contains(@class, 'reusable-search__result-container')]")
    for row in rows:
        try:
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", row)
            try:
                follow_button = WebDriverWait(row, 5).until(EC.element_to_be_clickable((By.XPATH, "(.//button[normalize-space(.)='Follow'])[1]")))
                state=1
            except:
                connect_button = WebDriverWait(row, 5).until(EC.element_to_be_clickable((By.XPATH, "(.//button[normalize-space(.)='Connect'])[1]")))
                state=2
            try: 
                if state == 1 or state == 2:
                    print(f"Count: {count}")
                    username_element = WebDriverWait(row, 5).until(EC.visibility_of_element_located((By.XPATH, f'(//*[@class="app-aware-link "])[{count}]')))
                    username = username_element.text
                    split_data = username.split('\n')
                    finalusername = split_data[0] if split_data else ""
                    print(f"User: {finalusername}")
                    save_user_in_csv("./UsersData/username.csv", finalusername)
                    designation = row.find_element(By.XPATH, ".//*[@class='entity-result__primary-subtitle t-14 t-black t-normal']").text
                    location = row.find_element(By.XPATH, ".//*[@class='entity-result__secondary-subtitle t-14 t-normal']").text
                    summary = row.find_element(By.XPATH, ".//p[contains(@class, 'entity-result__summary') and contains(@class, 'entity-result__summary--2-lines')]").text
                    split_data = summary.split(':')
                    current = split_data[-1] if split_data else ""
                    datalist.append({
                        "username": finalusername,
                        "designation": designation,
                        "location": location,
                        "current": current
                    })
                    count+=1
                else:
                    continue
            except:
                count+=1
                continue
                
        except Exception as e:
            count+=1
            print(f"Something Happened: {count}")
            continue
    return datalist

In [109]:
scrapdata(driver)

Count: 2
User: Ahmad Chaudhary
Something Happened: 4
Count: 4
User: Usman Ghani Khattak
Count: 5
User: Abdul karim
Count: 6
User: Waqas Arshad
Something Happened: 8
Something Happened: 9
Count: 9
User: Asad Ur Rehman
Something Happened: 11
Count: 11
User: AMMAR FAISAL AL ASSAD


[{'username': 'Ahmad Chaudhary',
  'designation': 'Founder & Chief Executive Officer (CEO) @ Smart IT Tec | Technology Consultant | Innovator | Building Technology for Companies Worldwide',
  'location': 'Lahore District',
  'current': ' Chief Executive Officer at Smart IT Tec'},
 {'username': 'Usman Ghani Khattak',
  'designation': 'CEO at SONY IT SUPPORT',
  'location': 'Pakistan',
  'current': ' CEO at SONY IT SUPPORT'},
 {'username': 'Abdul karim',
  'designation': 'Chief Executive Officer at OMECTA USA - Amazon Solutions Providers',
  'location': 'Islāmābād, Pakistan',
  'current': ' Managing Director at Center of excellence in networking & IT Private Limited.'},
 {'username': 'Waqas Arshad',
  'designation': 'CEO @BitBytes | Helping IT & Digital Companies Design, Develop & Deliver With Confidence 🔥',
  'location': 'Lahore District',
  'current': ' CEO at BitBytes'},
 {'username': 'Asad Ur Rehman',
  'designation': 'Founder & CEO @ Karobar Solutions LLC & Karobar Impact Fund | Rem

In [29]:
def is_login(driver):
    try:
        chklogin = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,searchbtnxpath)))
        return True
    except:
        return False
    

In [50]:
def main(driver, names , message):
    try:
        count = 0
        for name in names:
            print(f"1: {name}")
            autosearch(driver, name)
            time.sleep(5)
            if count == 0:
                findpeople(driver)
            datalist = scrapdata(driver)
            save_info(name, datalist)
            for _ in range(2):
                find_next(driver)
                print("Wait for Loading...")
                time.sleep(10)
                datalist = scrapdata(driver)
                time.sleep(5)
                save_info(name, datalist)
            count = 1
            time.sleep(5)
    except Exception as e:
        print(f"Unhandled Exception occurred: {e}")


In [ ]:
names = ["CEO of IT Companies" , "CTO of IT Companies"]
main(driver, names , message)

In [31]:
if __name__ == "__main__":
    names = ["CEO of IT Companies" , "CTO of IT Companies"]
    profileid='bbd273f6-08db-4cf3-95c9-88cfa4ee9060'
    message = "Hey! This is test message."
    max_attempts=3
    for attempt in range(max_attempts):
        try:
            driver = openprofile(profileid)
            if driver:
                break
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")

    print("Wait For Loading...")
    time.sleep(10)
    print("CHECK LOGIN STATUS....")
    status= is_login(driver)
    if status:
        print("You are already Logged in System...")
        print("Start Working...")
        main(driver , names , message)
    else:
        print("You are Not Logged in System...")
        print("Start Login...")
        autologin(driver , Email , passwd)
        print("Wait For Loading...")
        time.sleep(20)
        print("Start Working...")
        main(driver , names , message)
    print("All Working Done...")
    driver.quit()
    

Browser opened successfully with profile ID: bbd273f6-08db-4cf3-95c9-88cfa4ee9060
(25) Feed | LinkedIn is open !!!!
Wait For Loading...
CHECK LOGIN STATUS....
You are already Logged in System...
Start Working...
Unhandled Exception occurred....
All Working Done...


In [226]:
def send_message(driver, message):
    try:
        action = ActionChains(driver)
        msgbtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "(//button[contains(., 'Message')])[2]")))
        action.move_to_element(msgbtn)
        time.sleep(1)
        msgbtn.click()
        print("Message Btn Clicked")
        time.sleep(5)
        msgarea = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'msg-form__msg-content-container') and contains(@class, 'msg-form__message-texteditor')]//div[contains(@class, 'msg-form__msg-content-container--scrollable')]//div[contains(@class, 'msg-form__contenteditable')]")))
        action.move_to_element(msgarea)
        time.sleep(1)
        msgarea.click()
        print("Message Area Clicked")
        print("Type Message...")
        type_like_human(msgarea, message)
        sendbtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[text()='Send']")))
        action.move_to_element(sendbtn)
        time.sleep(3)
        # sendbtn.click()
        closebtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@class='msg-overlay-bubble-header__control artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view']")))
        action.move_to_element(closebtn)
        time.sleep(1)
        closebtn.click()
        time.sleep(2)
    except Exception as e:
        print(f"An error occurred while sending message:")
        try:
            closebtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@class='msg-overlay-bubble-header__control artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view']")))
            action.move_to_element(closebtn)
            time.sleep(1)
            closebtn.click()
            time.sleep(2)
        except Exception as e:
            print(f"An error occurred while closing the message window:")
def type_like_human(element, text):
    for character in text:
        element.send_keys(character)
        time.sleep(0.1)

In [223]:
def scrapdata(driver, message):
    count=2
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@class='search-results-container']")))
        container = driver.find_element(By.XPATH, "//*[@class='search-results-container']")
        rows = container.find_elements(By.XPATH, ".//li[contains(@class, 'reusable-search__result-container')]")
        datalist = []
        for row in rows:
            try:
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", row)
                userprofile = row.find_element(By.XPATH, f'(//*[@class="app-aware-link "])[{count}]')
                username = row.find_element(By.XPATH, f'(//*[@class="app-aware-link "])[{count}]').text
                split_data = username.split('\n')
                finalusername = split_data[0] if split_data else ""
                designation = row.find_element(By.XPATH, ".//*[@class='entity-result__primary-subtitle t-14 t-black t-normal']").text
                designation_text = designation.lower()
                if "it" in designation_text:
                    print("Desired designation text found!")
                    userprofile.click()
                    print("Wait for Loading...")
                    time.sleep(10)
                    send_message(driver, message)
                    print("Wait ...")
                    driver.back()  
                    time.sleep(5)
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@class='search-results-container']")))
                    container = driver.find_element(By.XPATH, "//*[@class='search-results-container']")
                    rows = container.find_elements(By.XPATH, ".//li[contains(@class, 'reusable-search__result-container')]")
                else:
                    print("Desired designation text not found.")
                    continue
                location = row.find_element(By.XPATH, ".//*[@class='entity-result__secondary-subtitle t-14 t-normal']").text
                summary = row.find_element(By.XPATH, ".//p[contains(@class, 'entity-result__summary') and contains(@class, 'entity-result__summary--2-lines')]").text
                split_data = summary.split(':')
                current = split_data[-1] if split_data else ""
                datalist.append({
                    "username": finalusername,
                    "designation": designation,
                    "location": location,
                    "current": current
                })
                count += 1
            except Exception as e:
                print(f"An error occurred while processing a row: ")
    except Exception as e:
        print(f"An error occurred while waiting for the search results container: ")
    
    return datalist

In [225]:
scrapdata(driver, message)

Desired designation text found!
Wait for Loading...
Message Btn Clicked
An error occurred while sending message: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF68F7CE232+316962]
	(No symbol) [0x00007FF68F625132]
	(No symbol) [0x00007FF68F331B89]
	(No symbol) [0x00007FF68F374F9F]
	(No symbol) [0x00007FF68F3751D0]
	(No symbol) [0x00007FF68F3AECF7]
	(No symbol) [0x00007FF68F3907DF]
	(No symbol) [0x00007FF68F36C3BA]
	(No symbol) [0x00007FF68F3ACCD7]
	(No symbol) [0x00007FF68F38FEC3]
	(No symbol) [0x00007FF68F36B328]
	(No symbol) [0x00007FF68F369991]
	(No symbol) [0x00007FF68F36A8F8]
	(No symbol) [0x00007FF68F315C15]
	GetHandleVerifier [0x00007FF68F801619+526857]
	sqlite3_dbdata_init [0x00007FF68FA3D3BB+687435]
	sqlite3_dbdata_init [0x00007FF68FA3C70C+684188]
	sqlite3_dbdata_init [0x00007FF68FA37CE1+665201]
	sqlite3_dbdata_init [0x00007FF68FA3E6C8+692312]
	(No symbol) [0x00007FF68F642D7D]
	(No symbol) [0x00007FF68F632F67]
	(No symbol) [0x00007FF68F6332F1]
	(No symbol) [0x00007FF68F60FE1

[]

In [205]:
profileid='bbd273f6-08db-4cf3-95c9-88cfa4ee9060'
message = "Hey! This is test message."
max_attempts=3
for attempt in range(max_attempts):
    try:
        driver = openprofile(profileid)
        if driver:
            break
    except Exception as e:
        print(f"Attempt {attempt + 1} failed: {str(e)}")

Browser opened successfully with profile ID: bbd273f6-08db-4cf3-95c9-88cfa4ee9060
Attempt 1 failed: 'NoneType' object has no attribute 'execute'
Browser opened successfully with profile ID: bbd273f6-08db-4cf3-95c9-88cfa4ee9060
(25) Feed | LinkedIn is open !!!!


In [ ]:
# for row in rows:
                #     driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", row)
                #     userprofile = row.find_element(By.XPATH, f'(//*[@class="app-aware-link "])[{count}]')
                #     username = userprofile.text
                #     split_data = username.split('\n')
                #     finalusername = split_data[0] if split_data else ""
                    
                #     designation = row.find_element(By.XPATH, ".//*[@class='entity-result__primary-subtitle t-14 t-black t-normal']").text
                #     designation_text = designation.lower()
                #     if "it" in designation_text:
                #         print("Desired designation text found!")
                #     else:
                #         print("Desired designation text not found.")
                #         count+=1
                #         continue
                    
                #     location = row.find_element(By.XPATH, ".//*[@class='entity-result__secondary-subtitle t-14 t-normal']").text
                #     summary = row.find_element(By.XPATH, ".//p[contains(@class, 'entity-result__summary') and contains(@class, 'entity-result__summary--2-lines')]").text
                #     split_data = summary.split(':')
                #     current = split_data[-1] if split_data else ""
                #     datalist.append({
                #         "username": finalusername,
                #         "designation": designation,
                #         "location": location,
                #         "current": current
                #     })
                #     userprofile.click()
                #     print("Wait for Loading...")
                #     time.sleep(10)
                #     send_message(driver, message)
                #     print("Wait ...")
                #     time.sleep(5)

In [123]:
get_usernames("./userfiles/CEO of IT Companies.txt", "./UsersData/username.csv")
# get_username("./userfiles/CEO of IT Companies.txt", "./UsersData/username.csv")

Ignoring line: 
Ignoring line: 


In [158]:
container = driver.find_element(By.XPATH, "//*[@class='search-results-container']")

In [161]:
len(container)

TypeError: object of type 'WebElement' has no len()

In [179]:
rows = driver.find_elements(By.XPATH, ".//li[contains(@class, 'reusable-search__result-container')]")

In [180]:
len(rows)

10

In [210]:
def send_message(driver, message):
    try:
        action = ActionChains(driver)
        try:
            connectbtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"(//span[text()='Connect']/ancestor::button)[2]")))
            action.move_to_element(connectbtn)
            time.sleep(1)
            connectbtn.click()
            print("Connect Button Clicked.")
        except:
            pass
        try:
            followbtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"(//span[text()='Follow']/ancestor::button)[2]")))
            action.move_to_element(followbtn)
            time.sleep(1)
            followbtn.click()
            print("Follow Button Clicked.")
        except:
            pass
        try:
            connectpopup = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"//span[text()='Send without a note']/ancestor::button")))
            action.move_to_element(connectpopup)
            time.sleep(1)
            connectpopup.click()
            print("Pop up Clicked...")
        except:
            pass

        time.sleep(5)

        msgbtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "(//button[contains(., 'Message')])[2]")))
        action.move_to_element(msgbtn)
        time.sleep(1)
        msgbtn.click()
        print("Message Btn Clicked")
        time.sleep(5)
        msgarea = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'msg-form__msg-content-container') and contains(@class, 'msg-form__message-texteditor')]//div[contains(@class, 'msg-form__msg-content-container--scrollable')]//div[contains(@class, 'msg-form__contenteditable')]")))
        action.move_to_element(msgarea)
        time.sleep(1)
        msgarea.click()
        print("Message Area Clicked")
        print("Type Message...")
        type_like_human(msgarea, message)
        sendbtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[text()='Send']")))
        action.move_to_element(sendbtn)
        time.sleep(3)
        # sendbtn.click()
        
        closebtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@class='msg-overlay-bubble-header__control artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view']")))
        action.move_to_element(closebtn)
        time.sleep(1)
        closebtn.click()
        time.sleep(2)

        discardbtn = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH ,"//button[normalize-space(.)='Discard']")))
        action.move_to_element(discardbtn)
        time.sleep(1)
        discardbtn.click()
        time.sleep(2)
    except Exception as e:
        print(f"An error occurred while sending message:")
        try:
            closebtn = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@class='msg-overlay-bubble-header__control artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view']")))
            action.move_to_element(closebtn)
            time.sleep(1)
            closebtn.click()
            time.sleep(2)
        except Exception as e:
            print(f"An error occurred while closing the message window:")

In [223]:
def userselection(driver):
    message = "hey, this is test messsage."    
    action = ActionChains(driver)
    rows = driver.find_elements(By.XPATH, ".//li[contains(@class, 'reusable-search__result-container')]")
    time.sleep(5)
    count=1
    for row in rows:
        
        row = driver.find_element(By.XPATH, f"(.//li[contains(@class, 'reusable-search__result-container')])[{count}]")
        print(f"{row.text}")
        action.move_to_element(row)
        time.sleep(1)
        row.click()
        try:
            dismiss = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"//span[text()='Got it']/ancestor::button")))
            if dismiss:
                print("Dismiss found!")
            action.move_to_element(dismiss)
            dismiss.click()
            time.sleep(5)
            count+=1
            continue
        except:
            pass
        time.sleep(5)
        print("Start Messaging....")
        send_message(driver, message)
        print("Now Back to Page...")
        time.sleep(5)
        driver.back()
        time.sleep(5)
        count+=1

In [224]:
userselection(driver)

LinkedIn Member
C.E.O & Managing Director At Z&S Group Of Companies
Sambrial
Current: CEO And Managing Director at Z&S IT Solutions (SMC-PVT) LTD.
Dismiss found!
LinkedIn Member
Founder
Punjab, Pakistan
Current: Chief Executive Officer at iVision IT Hub
Dismiss found!
LinkedIn Member
Founder & CEO at CAD-IT Group of Companies
Singapore
Current: Founder & CEO at CAD-IT Group of Companies - 1) Subsidiaries of CAD-IT Global Holdings Pte. Ltd. are: CAD-IT Pte. Ltd. CAD-IT Investments Pte...
Dismiss found!
LinkedIn Member
UI | UX Designer and seeking collaboration with software and design companies for Papertopixel in creating a technology empire that impacts change in the IT industry of Pakistan.
Lahore District
Current: Founder, CEO at Paper to Pixel
Dismiss found!
LinkedIn Member
Chief Executive Officer at IT Companies Network
Bend, OR
Dismiss found!
Sandeep Soni
View Sandeep Soni’s profile
• 3rd+
3rd+ degree connection
Chairman Founder & Group CEO at bGlide Group of Companies | Currentl